In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
def get_shape(tensor):
    "return the shape of tensor as list"
    return tensor.get_shape().as_list()

def print_var(string):
    "print var_name and var_shape in collection named string"
    print("\n".join(["{} : {}".format(v.name, get_shape(v)) for v in tf.get_collection(string)]))
    
def clip(x, vmax = 1-1e-7, vmin = 1e-7):
    "clip the x between vmax, vmin"
    return tf.clip_by_value(x, clip_value_max=vmax, clip_value_min=vmin)

def batch_norm(self, x, n_channel, phase_train, name):
    with tf.variable_scope(name):
        beta = tf.Variable(tf.constant(0.0, shape = [n_channel]), name = 'beta')
        gamma = tf.Variable(tf.constant(1.0, shape = [n_channel]), name = 'gamma')
        batch_mean, batch_var = tf.nn.moments(x, [0, 1, 2], name = 'moments')
        ema = tf.train.ExponentialMovingAverage(decay = 0.5)
        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var) 

        mean, var = tf.cond(tf.cast(phase_train, tf.bool), mean_var_with_update, lambda:(ema.average(batch_mean), ema.average(batch_var))) 
    normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

In [ ]:
mnist = input_data.read_data_sets('../MNIST_data', one_hot=True)

In [ ]:
train = {}
test = {}
val = {}
train['input'] = mnist.train.images
train['output'] = mnist.train.labels
test['input'] = mnist.test.images
test['output'] = mnist.test.labels
val['input'] = mnist.validation.images
val['output'] = mnist.validation.labels

In [ ]:
# Model parameter
image_size = 28*28
image_row = 28
conv_filter1 = [5, 5]
conv_channel1 = 30
conv_filter2 = [3, 3]
conv_channel2 = 50
fc_hidden = 500
nclasses = 10

# Train parameter
learning_rate = 1e-4
epoch = 5
ntrain = len(train['input'])
batch_size = 100
nbatch = ntrain//batch_size
log_every = 50

In [ ]:
x = tf.placeholder(dtype = tf.float32, shape = [None, image_size], name = 'image')
y = tf.placeholder(dtype = tf.float32, shape = [None, nclasses], name = 'label')

x_reshape = tf.reshape(x, [-1, image_row, image_row, 1]) # 4D 

conv_layer1 = {}
conv_layer2 = {}
fc_layer1 = {}
fc_layer2 = {}

with tf.variable_scope("conv_layer1"):
    conv_layer1['w'] = tf.Variable(initial_value = tf.random_normal(shape = [conv_filter1[0], conv_filter1[1], 1, conv_channel1], mean = 0.0, stddev = 0.1),
                              name = 'weight')
    conv_layer1['b'] = tf.Variable(initial_value = tf.constant(value=0.01, shape = [conv_channel1]),
                              name = 'bias')
    conv_layer1['conv'] = tf.nn.conv2d(x_reshape, conv_layer1['w'], strides = [1, 1, 1, 1], padding = "VALID") + conv_layer1['b']
    conv_layer1['relu'] = tf.nn.relu(conv_layer1['conv'])
    conv_layer1['pool'] = tf.nn.max_pool(conv_layer1['relu'], ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')
    conv_layer1['mean'], conv_layer1['var'] = tf.nn.moments(conv_layer1['pool'], [0, 1, 2])
    conv_layer1['norm'] = (conv_layer1['pool']-conv_layer1['mean'])/(conv_layer1['var']+1e-3)
    
with tf.variable_scope("conv_layer2"):
    conv_layer2['w'] = tf.Variable(initial_value = tf.random_normal(shape = [conv_filter2[0], conv_filter2[1], conv_channel1, conv_channel2], mean = 0.0, stddev = 0.1),
                              name = 'weight')
    conv_layer2['b'] = tf.Variable(initial_value = tf.constant(value=0.01, shape = [conv_channel2]),
                              name = 'bias')
    conv_layer2['conv'] = tf.nn.conv2d(conv_layer1['norm'], conv_layer2['w'], strides = [1, 1, 1, 1], padding = "VALID") + conv_layer2['b']
    conv_layer2['relu'] = tf.nn.relu(conv_layer2['conv'])
    conv_layer2['pool'] = tf.nn.max_pool(conv_layer2['relu'], ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID')
    conv_layer2['mean'], conv_layer2['var'] = tf.nn.moments(conv_layer2['pool'], [0, 1, 2])
    conv_layer2['norm'] = (conv_layer2['pool']-conv_layer2['mean'])/(conv_layer2['var']+1e-3)
    conv_shape = get_shape(conv_layer2['norm'])
    flat_size = conv_shape[1]*conv_shape[2]*conv_shape[3]
    conv_layer2['flat'] = tf.reshape(conv_layer2['norm'], [-1, flat_size])    
    
with tf.variable_scope("fc_layer1"):
    fc_layer1['w'] = tf.Variable(initial_value = tf.random_normal(shape = [flat_size, fc_hidden], mean = 0.0, stddev = 0.1),
                              name = 'weight')
    fc_layer1['b'] = tf.Variable(initial_value = tf.constant(value=0.01, shape = [fc_hidden]),
                              name = 'bias')
    fc_layer1['output'] = tf.nn.relu(tf.add(tf.matmul(conv_layer2['flat'], fc_layer1['w']), fc_layer1['b']))    

with tf.variable_scope("fc_layer2"):
    fc_layer2['w'] = tf.Variable(initial_value = tf.random_normal(shape = [fc_hidden, nclasses], mean = 0.0, stddev = 0.1),
                              name = 'weight')
    fc_layer2['b'] = tf.Variable(initial_value = tf.constant(value=0.01, shape = [nclasses]),
                              name = 'bias')
    fc_layer2['output'] = tf.nn.softmax(tf.add(tf.matmul(fc_layer1['output'], fc_layer2['w']), fc_layer2['b']))    
    
print("Reshape {} -> {}".format(get_shape(x), get_shape(x_reshape)))
print("Convolution layer 1")
print("Conv {} -> {}".format(get_shape(x_reshape), get_shape(conv_layer1['conv'])))
print("Pool {} -> {}".format(get_shape(conv_layer1['conv']), get_shape(conv_layer1['pool'])))    
print("Convolution layer 2")
print("Conv {} -> {}".format(get_shape(conv_layer1['pool']), get_shape(conv_layer2['conv'])))
print("Pool {} -> {}".format(get_shape(conv_layer2['conv']), get_shape(conv_layer2['pool'])))   
print("Flat {} -> {}".format(get_shape(conv_layer2['pool']), get_shape(conv_layer2['flat'])))
print("Fully connected layer 1")
print("Linear {} -> {}".format(get_shape(conv_layer2['flat']), get_shape(fc_layer1['output'])))
print("Fully connected layer 2")
print("Linear {} -> {}".format(get_shape(fc_layer1['output']), get_shape(fc_layer2['output'])))

In [ ]:
print_var("trainable_variables")

In [ ]:
cross_entropy = -tf.reduce_mean(y*tf.log(clip(fc_layer2['output'])))

correct_prediction = tf.equal(tf.argmax(fc_layer2['output'], 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

run_train = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
train_accuracy_tract = []
val_accuracy_tract = []

for epoch_ in range(epoch):
    index = np.arange(len(train['input']))
    np.random.shuffle(index)
    trX = train['input'][index]
    trY = train['output'][index]
    val_feed_dict = {x : val['input'], 
                     y : val['output']}
    
    for nbatch_ in range(nbatch):
        train_feed_dict = {x : trX[batch_size*nbatch_:batch_size*(nbatch_+1)], 
                           y : trY[batch_size*nbatch_:batch_size*(nbatch_+1)]}
        sess.run(run_train, feed_dict=train_feed_dict)
        
        if nbatch_%log_every == log_every - 1:
            train_accuracy = 100*sess.run(accuracy, feed_dict=train_feed_dict)
            val_accuracy = 100*sess.run(accuracy, feed_dict= val_feed_dict)
            train_accuracy_tract.append(train_accuracy)
            val_accuracy_tract.append(val_accuracy)
            print("Epoch(%d/%d) %d th batch train : %.1f %%, validation : %.1f %%"%(epoch_+1, epoch, nbatch_+1, train_accuracy, val_accuracy))

test_feed_dict = {x : test['input'],  
                  y : test['output']}

test_accuracy = 100*sess.run(accuracy, feed_dict=test_feed_dict)
print("Test accuracy=%.1f %%"%(test_accuracy))

plt.plot(train_accuracy_tract, 'r-', label = 'Train accuracy')
plt.plot(val_accuracy_tract, 'b-', label = 'Validation accuracy')
plt.legend()
plt.show()